<a href="https://colab.research.google.com/github/fatimazain118/SVM-Kernels-Stacking-Ensemble/blob/main/Stacking_greenstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
Abhi = pd.read_csv("/content/gdrive/MyDrive/greenstone 3.0.7.csv")

In [ ]:
Abhi['ALTER'] = Abhi['ALTER'].map({'YES': 1, 'NO': 0})

In [ ]:
Abhi.shape

(601, 16)

In [ ]:
Abhi.head()

,CountClassCoupled,CountClassDerived,CountDeclInstanceMethod,CountDeclInstanceVariable,CountDeclMethodAll,CountDeclMethodPrivate,CountDeclMethodPublic,CountLineCode,MaxInheritanceTree,PercentLackOfCohesion,SumCyclomatic,ADDED,DELETED,MODIFIED,TOTAL CHANGE,ALTER
0,4,0,30,6,34,12,19,189,1,78,24,8,0,0,8,1
1,6,0,5,2,5,0,5,133,1,50,20,21,4,1,26,1
2,7,0,0,0,60,55,5,2124,1,0,355,73,129,20,222,1
3,21,0,6,0,15,1,14,145,1,85,17,0,0,10,10,1
4,10,0,13,6,13,7,6,721,1,91,27,8,0,1,9,1


In [ ]:
X = Abhi.drop('ALTER', axis=1)
y = Abhi['ALTER']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('sig', SVC(probability=True,kernel='sigmoid')))
	level0.append(('lin', SVC(probability=True,kernel='linear')))
	level0.append(('poly', SVC(probability=True,kernel='poly')))
	level0.append(('rbf', SVC(probability=True,kernel='rbf')))
	# define meta learner model
	level1 = SVC(probability=True,kernel='rbf')
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1)
	return model

In [ ]:
def get_models():
	models = dict()
	#models['lin'] = SVC(kernel='linear')
	#models['sig'] = SVC(kernel='sigmoid')
	#models['poly'] = SVC(kernel='poly')
	#models['rbf'] = SVC(kernel='rbf')

	models['stacking'] = get_stacking()
	return models

In [ ]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [ ]:
models = get_models()

In [ ]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))


>stacking 0.207 (0.405)


In [ ]:
print(y_pred)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 0]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[92  0]
 [ 4 25]]


In [ ]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
1.0


In [ ]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.9089276237893511


In [ ]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.9310344827586207


In [ ]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.9284766908852593


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        92
           1       1.00      0.86      0.93        29

    accuracy                           0.97       121
   macro avg       0.98      0.93      0.95       121
weighted avg       0.97      0.97      0.97       121

